In [1]:
import pickle
import re
from funcs import *

pdf_pickle = open("google_results_pdf.pkl", "rb")

pdf_urls = sorted(
    list(set([res["items"][0]["link"] for res in pickle.load(pdf_pickle)]))
)
print(len(pdf_urls))
pdf_urls

992


['http://app.kutztown.edu/policyregister/policy.aspx?policy=ACA-069',
 'http://bulletin.oglethorpe.edu/files/2017/08/2017-2018-Bulletin.pdf',
 'http://catalog.csp.edu/pdf/2020-2021.pdf',
 'http://catalog.greenville.edu/en/2010-2012-Undergraduate-Catalog',
 'http://catalog.husson.edu/archives/2014-15/pdf/2014-15.pdf',
 'http://catalog.missouriwestern.edu/archives/catalog1112.pdf',
 'http://catalog.mst.edu/graduate/generalinformation/informationtechnologyit/informationtechnologyit.pdf',
 'http://catalog.pba.edu/mime/media/view/23/1833/2016-17+PBA_Articulation_Manual+FINAL.pdf',
 'http://catalog.swu.edu/mime/media/6/671/2014-15-swu-undergraduate-bulletin.pdf',
 'http://catalog.uwgb.edu/archive/2016-17-graduate.pdf',
 'http://cis.bentley.edu/htopi/MSIS2006.pdf',
 'http://cit.suny.edu/wp-content/uploads/2021/03/CIT-2017-Book.pdf',
 'http://cq5publish.ou.edu/content/dam/IT/cybersecurity/Docs/policies-standards-and-guidelines/all-users/University_wide_active_policies/ou-it-information-protect

In [2]:
pdf_urls_filtered = negative_filter(pdf_urls)
print(len(pdf_urls_filtered))
pdf_urls_filtered

417


['http://app.kutztown.edu/policyregister/policy.aspx?policy=ACA-069',
 'http://cis.bentley.edu/htopi/MSIS2006.pdf',
 'http://cq5publish.ou.edu/content/dam/IT/cybersecurity/Docs/policies-standards-and-guidelines/all-users/University_wide_active_policies/ou-it-information-protection-policy-2020.pdf',
 'http://cs.lewisu.edu/mathcs/msis/projects/msis595_FelindaHolmes.pdf',
 'http://digitalcommons.georgiasouthern.edu/cgi/viewcontent.cgi?article=1034&context=infotech',
 'http://digitalcommons.usm.maine.edu/cgi/viewcontent.cgi?article=1013&context=cepare_technology',
 'http://elpaso.ttuhsc.edu/opp/_documents/56/op5601.pdf',
 'http://fye.wpi.edu/files/2015/04/Network-Information.pdf',
 'http://help.bw.edu/assets/policy/Policies/ITP-BW-01-AcceptableUsePolicy.pdf',
 'http://help.cedarcrest.edu/pdf/Office%20of%20Information%20Technology%20Acceptable%20Use%20Policy.pdf',
 'http://mit.spelman.edu/documents/FacultyTechnologyGuide-17-18A.pdf',
 'http://pdl.cmu.edu/Retreat/MonthlyWirelessAccessCodeand

In [3]:
pdf_urls_positive = positive_filter(pdf_urls_filtered, pdf=True)
print(len(pdf_urls_positive))
pdf_urls_positive

126


['http://fye.wpi.edu/files/2015/04/Network-Information.pdf',
 'http://help.bw.edu/assets/policy/Policies/ITP-BW-01-AcceptableUsePolicy.pdf',
 'http://help.cedarcrest.edu/pdf/Office%20of%20Information%20Technology%20Acceptable%20Use%20Policy.pdf',
 'http://pdl.cmu.edu/Retreat/MonthlyWirelessAccessCodeandInstructionsforSVD2018.pdf',
 'http://sacs.uno.edu/compliance-certification/docs/Campus_Wireless_Map.pdf',
 'http://security.marist.edu/aup.pdf',
 'http://tech.rochester.edu/wp-content/uploads/acceptable-use-policy.pdf',
 'http://www.colby.edu/its/wp-content/uploads/sites/145/2019/10/Acceptable-Use-Policy-2019.pdf',
 'http://www.desu.edu/sites/flagship/files/document/31/08-07_acceptableusepolicy.pdf',
 'http://www.fvsu.edu/wp-content/uploads/2011/09/Acceptable-Use-Policy.pdf',
 'http://www.memphis.edu/umtech/solutions/wireless/docs/uofm_securefin2.pdf',
 'http://www.menlo.edu/wp-content/uploads/2015/02/acceptable-use-policy-it-services.pdf',
 'http://www.sfasu.edu/policies/wireless-netwo

In [4]:
import requests
import tempfile
import PyPDF2
import nltk
import random

test_urls = random.sample(pdf_urls_positive, 2)
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
test_urls

['https://txwes.edu/media/twu-training/Network-Protection-and-Info-Security-Policy_10302015.pdf',
 'https://www.berea.edu/iss/wp-content/uploads/sites/133/2016/11/Network-Guidelines-PDF-public.pdf']

In [21]:
import os

try:
    os.makedirs("AUPs/PDF")
except FileExistsError:
    pass

for url in pdf_urls_positive:
    print(url)
    school = re.search(r"(?<=[\./])\.*\w+\.edu", url).group(0)
    try:
        r = requests.get(url)
    except requests.exceptions.SSLError:
        pass
    f = tempfile.TemporaryFile()
    f.write(r.content)
    reader = None
    try:
        reader = PyPDF2.PdfFileReader(f)
        fixed_sents = []
        for p in range(reader.getNumPages()):
            page = reader.getPage(p)
            text = page.extractText()
            text = re.sub(r"\n+", "", text)
            # print(text)
            sentences = tokenizer.tokenize(text)
            fixed_sents.extend(fix_sentences(sentences))
        # print("\n\n\n-----FIXED-----\n\n\n")
        # print(*fixed_sents,sep="\n\n")
        with open("AUPs/PDF/" + school + ".txt", "w") as outfile:
            print(school)
            for s in fixed_sents:
                outfile.write(s + "\n")
    except Exception as e:
        print(e)
        print("+++++ BAD PDF +++++")
        continue
    print("------------------------------")

http://fye.wpi.edu/files/2015/04/Network-Information.pdf
wpi.edu
------------------------------
http://help.bw.edu/assets/policy/Policies/ITP-BW-01-AcceptableUsePolicy.pdf
bw.edu
------------------------------
http://help.cedarcrest.edu/pdf/Office%20of%20Information%20Technology%20Acceptable%20Use%20Policy.pdf
cedarcrest.edu
------------------------------
http://pdl.cmu.edu/Retreat/MonthlyWirelessAccessCodeandInstructionsforSVD2018.pdf
cmu.edu
------------------------------
http://sacs.uno.edu/compliance-certification/docs/Campus_Wireless_Map.pdf
uno.edu
------------------------------
http://security.marist.edu/aup.pdf
File has not been decrypted
+++++ BAD PDF +++++
http://tech.rochester.edu/wp-content/uploads/acceptable-use-policy.pdf
rochester.edu
------------------------------
http://www.colby.edu/its/wp-content/uploads/sites/145/2019/10/Acceptable-Use-Policy-2019.pdf
EOF marker not found
+++++ BAD PDF +++++
http://www.desu.edu/sites/flagship/files/document/31/08-07_acceptableusepol

berry.edu
------------------------------
https://www.centenaryuniversity.edu/wp-content/uploads/2021/01/OIT-2019-10-Network-Use-Policy.pdf
centenaryuniversity.edu
------------------------------
https://www.clarion.edu/about-clarion/policies/cu-policies/acceptable-use-of-technology-resources-policy.pdf
clarion.edu
------------------------------
https://www.coloradomesa.edu/information-technology/documents/2020-colorado-mesa-university-network-use-agreement.pdf
coloradomesa.edu
------------------------------
https://www.cs.virginia.edu/wiki/lib/exe/fetch.php?media=acceptableusepolicy.pdf
virginia.edu
------------------------------
https://www.davidsondavie.edu/wp-content/uploads/2021/08/Policy-7.2-Internet-and-Network-Acceptable-Use.pdf
davidsondavie.edu
------------------------------
https://www.depauw.edu/files/resources/unauthorizednetworkdevicespolicy-dec2018-2.pdf
depauw.edu
------------------------------
https://www.emich.edu/it/about/docs/policy/pol-62-wireless-com-20050901.pdf
EO

wku.edu
------------------------------
https://www.xavier.edu/ts/faculty/documents/acceptable-use-policy.pdf
xavier.edu
------------------------------
https://www2.cortland.edu/offices/information-resources/pdf/policies/InformationTechnologyResourcesAcceptableUsePolicy.pdf
cortland.edu
------------------------------
https://www2.lib.utexas.edu/d7/sites/default/files/vprovost/Computer-Software-and-Network-Use-Policy.pdf
utexas.edu
------------------------------
https://www2.morgan.edu/Documents/ADMINISTRATION/regents/docs/bor_acceptableuse_itresources.pdf
morgan.edu
------------------------------
